In [14]:
!pip install pysam

import pysam


## Filtering vcf files READS(AO)≥ 10 and VAF(AD) ≥ 1%
## Lim et al., 2021
vcf_files = ["SRR13974119_freebayes.vcf", "SRR13974120_freebayes.vcf"]

for file in vcf_files:
    input_vcf = pysam.VariantFile(file, "r")
    output_file = file.replace(".vcf", "_filtered.vcf")
    output_vcf = pysam.VariantFile(output_file, "w", header=input_vcf.header)

    for record in input_vcf:
        # Check if AD (allele depth) is available
        sample = record.samples[0]
        if 'AD' not in sample or sample['AD'] is None:
            continue

        ad = sample['AD']
        if len(ad) < 2:
            continue  # skip if no alternate allele

        ref_reads = ad[0]
        alt_reads = ad[1]
        total_reads = sum(ad)

        vaf = alt_reads / total_reads if total_reads > 0 else 0

        if alt_reads >= 10 and vaf >= 0.01:
            output_vcf.write(record)

    output_vcf.close()
    print(f"Filtered VCF saved as: {output_file}")


Filtered VCF saved as: SRR13974119_freebayes_filtered.vcf
Filtered VCF saved as: SRR13974120_freebayes_filtered.vcf


In [19]:
import pysam

#Checking the # of variant in original and filtered vcf files.
def count_variants(vcf_file):
    vcf = pysam.VariantFile(vcf_file)
    return sum(1 for _ in vcf)

for file in ["SRR13974119_freebayes.vcf", "SRR13974120_freebayes.vcf"]:
    filtered_file = file.replace(".vcf", "_filtered.vcf")
    print(f"{file}: {count_variants(file)} variants")
    print(f"{filtered_file}: {count_variants(filtered_file)} variants")


SRR13974119_freebayes.vcf: 324425 variants
SRR13974119_freebayes_filtered.vcf: 7509 variants
SRR13974120_freebayes.vcf: 202194 variants
SRR13974120_freebayes_filtered.vcf: 6746 variants


In [23]:

# Subsetting KRAS variants based on the position of the gene
vcf_files = ["SRR13974119_freebayes_filtered.vcf", "SRR13974120_freebayes_filtered.vcf"]
output_files = ["SRR13974119_freebayes_filtered_KRAS.vcf", "SRR13974120_freebayes_filtered_KRAS.vcf"]

#KRAS Position
kras_chr = "chr12"
kras_start = 25209795
kras_end   = 25245384

for vcf_file, out_file in zip(vcf_files, output_files):
    vcf_in = pysam.VariantFile(vcf_file)
    vcf_out = pysam.VariantFile(out_file, "w", header=vcf_in.header)

    for record in vcf_in:
        if record.chrom == kras_chr and kras_start <= record.pos <= kras_end:
            vcf_out.write(record)

    vcf_out.close()
    print(f"Filtered KRAS variants written to {out_file}")



Filtered KRAS variants written to SRR13974119_freebayes_filtered_KRAS.vcf
Filtered KRAS variants written to SRR13974120_freebayes_filtered_KRAS.vcf


In [22]:
import pysam

pre_file  = "SRR13974119_freebayes_filtered_KRAS.vcf"
post_file = "SRR13974120_freebayes_filtered_KRAS.vcf"

def get_variant_dict(vcf_file):
    ##Return a dictionary with variant key -> sample info (REF, ALT, AD, VAF)
    vcf = pysam.VariantFile(vcf_file)
    variant_dict = {}
    for r in vcf:
        key = (r.chrom, r.pos, r.ref, tuple(r.alts))
        sample = r.samples[0]  # assuming single sample
        ad = sample['AD']
        if len(ad) >= 2:
            ref_reads = ad[0]
            alt_reads = ad[1]
            total_reads = sum(ad)
            vaf = alt_reads / total_reads if total_reads > 0 else 0
            variant_dict[key] = {
                'REF': r.ref,
                'ALT': r.alts,
                'REF_READS': ref_reads,
                'ALT_READS': alt_reads,
                'VAF': vaf
            }
    return variant_dict

pre_variants  = get_variant_dict(pre_file)
post_variants = get_variant_dict(post_file)

# Find variants present in pre but absent in post
unique_pre = set(pre_variants.keys()) - set(post_variants.keys())
print(f"Variants present in pre-treatment but absent in post-treatment: {len(unique_pre)}\n")

# Print detailed info
for var in sorted(unique_pre):
    info = pre_variants[var]
    print(f"Chromosome: {var[0]}, Position: {var[1]}, REF: {info['REF']}, ALT: {','.join(info['ALT'])}, "
          f"REF_READS: {info['REF_READS']}, ALT_READS: {info['ALT_READS']}, VAF: {info['VAF']:.2%}")


Variants present in pre-treatment but absent in post-treatment: 2

Chromosome: chr12, Position: 25215659, REF: ATTTTTTTTTTTTTAAAC, ALT: ATTTTTTTTTTTTAAAC, REF_READS: 110, ALT_READS: 57, VAF: 34.13%
Chromosome: chr12, Position: 25250357, REF: A, ALT: C, REF_READS: 277, ALT_READS: 17, VAF: 5.78%
